<a href="https://colab.research.google.com/github/nathan-barry/ai2-cartography-reimplementation/blob/main/snli_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai datasets

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datasets

In [ ]:
# Load the SNLI dataset
dataset = datasets.load_dataset('snli')

# Get the first 20 examples
examples = dataset['train']

# Convert the examples to a pandas DataFrame
# df = pd.DataFrame(examples)

In [ ]:
# Count number of words in dataset
words_in_train = 0
words_in_test = 0
for example in dataset['train']:
  words_in_train += len(example['premise']) + len(example['hypothesis'])
for example in dataset['test']:
  words_in_test += len(example['premise']) + len(example['hypothesis'])
print(f"Words in train: {words_in_train}")
print(f"Words in test: {words_in_test}")

In [ ]:
print((words_in_train / 1000) * .002)

In [ ]:
# Check to see if dataset is in contiguous sets of 3 same prompts
for i in range(0, examples.num_rows, 3): 
    if examples[i]['premise'] != examples[i+1]['premise'] != examples[i+2]['premise']:
        print(f"Not 3 contiguous at i={i}")
        print(examples[i]['premise'])
        print(examples[i+1]['premise'])
        print(examples[i+2]['premise'])
print("Done")

In [ ]:
num_entailment = 0
num_neutral = 0
num_contradiction = 0
num_nan = 0

import matplotlib.pyplot as plt

# Count how many examples with each label appears (is the dataset balanced)
for ex in examples:
    if ex['label'] == 0:
        num_entailment += 1
    elif ex['label'] == 1:
        num_neutral += 1
    elif ex['label'] == 2:
        num_contradiction += 1
    else:
        num_nan += 1
    
print(num_entailment)
print(num_neutral)
print(num_contradiction)
print(num_nan)

# Create the pie chart
labels = ['Entailment', 'Neutral', 'Contradiction', 'NaN']
sizes = [num_entailment, num_neutral, num_contradiction, num_nan]
colors = ['green', 'yellow', 'red', 'gray']
plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%')
plt.title('Pie Chart of 4 Variables')
plt.show()

In [ ]:
# FINDING: THIS IS FLAWED. SOME PREMISES REPEAT 30+ TIMES
num_good = 0
num_repeat = 0
num_bad = 0

# How many examples have all 3 labels (0,1,2), no repeats like (1,2,2)
for i in range(0, examples.num_rows, 3): 
    if (examples[i]['label'] == -1 or examples[i+1]['label'] == -1 or examples[i+2]['label'] == -1):
        num_bad += 1
    elif (examples[i]['label'] != examples[i+1]['label'] != examples[i+2]['label']):
        num_good += 1
    else:
        num_repeat += 1
print(num_good)
print(num_repeat)
print(num_bad)
print(3 * (num_good + num_repeat + num_bad))

# Create the pie chart
labels = ['good', 'repeat', 'nan']
sizes = [num_good, num_repeat, num_bad]
plt.pie(sizes, labels=labels, autopct='%1.1f%%')
plt.show()

In [ ]:
# How many examples have all 3 labels (0,1,2), no repeats like (1,2,2)
missing_labels = []
for i in range(0, examples.num_rows, 3): 
    seen = set([examples[i]['label'], examples[i+1]['label'], examples[i+2]['label']])
    for num in range(3):
        if num not in seen:
            missing_labels.append({'premise': examples[i]['premise'], 'label': num})

print(f"num of missing labels: {len(missing_labels)}\nnum of bad + repeat labels: {num_bad + num_repeat}")
for example in missing_labels[:10]:
    print(example)

In [ ]:
def get_premise_dict(examples):
    premise_dict = {}
    for entry in examples: 
        if entry['premise'] not in premise_dict:
            premise_dict[entry['premise']] = 1
        else:
            premise_dict[entry['premise']] += 1
    return premise_dict

premise_dict = get_premise_dict(examples)

In [ ]:
# FINDING: some premises are used a lot!
print(len(premise_dict)) # This should == last line if premises aren't reused
print(len(examples))
print(len(examples)/3)

In [ ]:
print(f"Sum of values: {sum(premise_dict.values())}") # should be 550152
print(f"Max value: {max(premise_dict.values())}")
print(f"Min value: {min(premise_dict.values())}") # should be 3
print(f"Num of premises, more than 3 ex: {len([s for s in filter(lambda x: x > 3, premise_dict.values())])}")
print(f"Num of premises, 3 ex: {len([s for s in filter(lambda x: x == 3, premise_dict.values())])}")
print(f"Num of premises, total: {len(premise_dict.values())}")

In [ ]:
print([s for s in filter(lambda x: x[1] == 33, premise_dict.items())]) # The offending premise

In [ ]:
res = df[df["premise"] == "A dog in a field."]
res.head(20)

In [ ]:
# PRINT STATS OF PREMISES THAT REPEAT MORE THAN 3 TIMES
my_list = [s for s in filter(lambda x: x > 3, premise_dict.values())]

# Calculate the mean, median, standard deviation, and variance
mean = np.mean(my_list)
median = np.median(my_list)
std_dev = np.std(my_list)
variance = np.var(my_list)

# Print the results
print(f"Mean: {mean}")
print(f"Median: {median}")
print(f"Standard Deviation: {std_dev}")
print(f"Variance: {variance}")

# Plot the distribution using a histogram
plt.hist(my_list, bins=10)
plt.xlabel("Values")
plt.ylabel("Frequency")
plt.title("Distribution of Values")
plt.show()


In [ ]:
# PRINT STATS ALL PREMISES
my_list = list(premise_dict.values())

# Calculate the mean, median, standard deviation, and variance
mean = np.mean(my_list)
median = np.median(my_list)
std_dev = np.std(my_list)
variance = np.var(my_list)

# Print the results
print(f"Mean: {mean}")
print(f"Median: {median}")
print(f"Standard Deviation: {std_dev}")
print(f"Variance: {variance}")

# Plot the distribution using a histogram
plt.hist(my_list, bins=10)
plt.xlabel("Values")
plt.ylabel("Frequency")
plt.title("Distribution of Values")
plt.show()

In [ ]:
# REAL NUMBER OF MISSING LABELS
def find_missing_labels(data):
    missing_labels = []
    premise_dict = {}

    for entry in data:
        premise = entry['premise']
        label = entry['label']

        if premise not in premise_dict:
            premise_dict[premise] = set()

        premise_dict[premise].add(label)

    for premise, labels in premise_dict.items():
        missing = {0, 1, 2} - labels
        if missing:
            for m in missing:
                missing_labels.append({'premise': premise, 'missing_label': m})

    return missing_labels

missing_labels = find_missing_labels(examples)
print(len(missing_labels))

In [ ]:
for label in missing_labels[:15]:
    print(label)